<a href="https://colab.research.google.com/github/samperochon/Rakuten_MultiModal_Color_Exctraction/blob/master/Bert_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
test=1